In [1]:
import pandas as pd
import time as t
from tqdm import tqdm

In [2]:
def addColon(row):
    # print(row, type(row))
    if(":" not in row):
        # print("Adding colon to", row)
        row = row[:-2] +":"+ row[-2:]
        return row
    else:
        return row

In [3]:
df = pd.read_csv("time-log-formatted.csv")
df = df[["Date", "Starttime", "Endtime", "Activity"]]
print(df.head(2))
df["Date"] = pd.to_datetime(df["Date"], format='%m/%d/%y')
print("Done")
# Had to sort starttime in excel, then scroll to bottom
df["Starttime"] = df["Starttime"].apply(lambda row: addColon(row))
df["Endtime"] = df["Endtime"].apply(lambda row: addColon(row))
# print(df["Starttime"], "\n", df["Endtime"])
df["Starttime"] = pd.to_timedelta(df["Starttime"]+":00")   # https://stackoverflow.com/a/49144483/14717416
# searched for "-05 ", ... , "-55"
df["Endtime"] = pd.to_timedelta(df["Endtime"]+":00")
print(df)
print(df.info())

       Date Starttime Endtime   Activity
0  11/11/20      8:30    9:30      break
1  11/11/20      9:30   10:20  breakfast
Done
            Date       Starttime         Endtime   Activity
0     2020-11-11 0 days 08:30:00 0 days 09:30:00      break
1     2020-11-11 0 days 09:30:00 0 days 10:20:00  breakfast
2     2020-11-11 0 days 10:20:00 0 days 10:45:00  breakfast
3     2020-11-11 0 days 10:45:00 0 days 12:20:00   planning
4     2020-11-11 0 days 12:20:00 0 days 12:30:00        108
...          ...             ...             ...        ...
14216 2022-04-30 0 days 20:15:00 0 days 21:50:00      break
14217 2022-04-30 0 days 21:50:00 0 days 22:20:00      break
14218 2022-04-30 0 days 22:20:00 0 days 23:05:00      break
14219 2022-04-30 0 days 23:05:00 0 days 23:15:00        162
14220 2022-04-30 0 days 23:15:00 0 days 23:55:00      break

[14221 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14221 entries, 0 to 14220
Data columns (total 4 columns):
 #   Column     No

#### TODO: Separate combined time blocks to 0.66/0.33 and 0.5/0.33/0.17

In [4]:
# First, get valid activities and details
# Separate them into groups: gen. activities, gen. classes, activities that should be details
all_vals = df["Activity"].unique()
print(all_vals, "\n")
activities = ['break','breakfast','planning','lunch','career',
              'exercise','dinner','sleep','appt','help','prep',
              'health','plan','bike','walk','bus']
print(set(all_vals) - set(activities), "\n")
classes = ['UDCE', '141b', '104', '135', '108', '137', '162', '171', '122a', '174', '131a', '168','167', 'CLA', 'fys', '330', '154a', '141a', '302', '130', '141c', '165a', '131c', '160', '141', '131b', '010', '106']
print("To replace:", set(all_vals) - set(activities) - set(classes))
addActivity = ['nap', 'OIT', 'drive', 'games', 'research', 'outing', 'TV', 'talk', "health", "internship", "sleep", "plan"]

['break' 'breakfast' 'planning' '108' 'lunch' 'games' '131a' 'career'
 'exercise' 'dinner' 'CLA' 'TV' 'fys' 'appt' 'nap' 'help' 'prep' 'talk'
 '165a' '141a' '131b' '154a' 'UDCE' '302' 'OIT' '141b' 'health' 'plan'
 '171' '106' '137' 'internship' '167' 'bike' '168' 'walk' '131c' '010'
 '122a' 'outing' 'research' 'drive' 'bus' '135' '104' '141c' '130' '330'
 '141' '162' '160' '174'] 

{'137', '104', '135', '141c', 'drive', '141', '141b', 'internship', 'games', '131c', '131b', 'UDCE', 'OIT', 'outing', 'research', '162', '141a', 'nap', '108', '330', '171', '165a', 'fys', '302', 'talk', '131a', '168', '106', 'TV', '010', '130', 'CLA', '160', '154a', '122a', '174', '167'} 

To replace: {'talk', 'OIT', 'outing', 'TV', 'drive', 'internship', 'games', 'nap', 'research'}


In [5]:
for x in range(0,4):
    print(5-x, 4-x)

5 4
4 3
3 2
2 1


In [6]:
for i in activities:
    print(i)

break
breakfast
planning
lunch
career
exercise
dinner
sleep
appt
help
prep
health
plan
bike
walk
bus


In [7]:
## Correct the addActivity rows so they all have an associated activity
# Assign the correct 
addActDict = {'nap':"break", 'OIT':"appt", 'drive':"break", 'games':"break", 
              'research':"career", 'outing':"break", 'TV':"break", 
              'talk':"break", "health":"appt", "internship":"career", 
              "sleep":"break", "plan":"planning"}
"""def addAct(row):
    print(row)
    #if(row["Activity"] in addActivity):
    if(row in addActivity):
        # Shift all the details right, truncate end and manually add Det1
        # for i in range(0, 4):
            # row["Det"+str(5-i)] = row["Det"+str(4-i)]
        # row["Det1"] = row["Activity"]  ## TEMP. DISABLED
        # Reassign Activity
        return addActDict[row["Activity"]]

df["Activity"] = df["Activity"].apply(lambda row: addAct(row))"""
df['Activity'] = df['Activity'].replace(addActDict)
print(df)
print("df:",df)
print(set(df["Activity"].unique()) - set(activities) - set(classes))
# for i in range(df.shape[0]):
#     print(i, "\n", sep="")
#     # row["Activity"] = "test value"  # does not change original
#     # print(row["Activity"], "\n")
#     print("Row by reference:\n", df.iloc[i,:])
#     df.iloc[i,:].Det5 = df.iloc[i,:].Activity
#     print("After alteration:\n", df.iloc[i,:])
#     break

            Date       Starttime         Endtime   Activity
0     2020-11-11 0 days 08:30:00 0 days 09:30:00      break
1     2020-11-11 0 days 09:30:00 0 days 10:20:00  breakfast
2     2020-11-11 0 days 10:20:00 0 days 10:45:00  breakfast
3     2020-11-11 0 days 10:45:00 0 days 12:20:00   planning
4     2020-11-11 0 days 12:20:00 0 days 12:30:00        108
...          ...             ...             ...        ...
14216 2022-04-30 0 days 20:15:00 0 days 21:50:00      break
14217 2022-04-30 0 days 21:50:00 0 days 22:20:00      break
14218 2022-04-30 0 days 22:20:00 0 days 23:05:00      break
14219 2022-04-30 0 days 23:05:00 0 days 23:15:00        162
14220 2022-04-30 0 days 23:15:00 0 days 23:55:00      break

[14221 rows x 4 columns]
df:             Date       Starttime         Endtime   Activity
0     2020-11-11 0 days 08:30:00 0 days 09:30:00      break
1     2020-11-11 0 days 09:30:00 0 days 10:20:00  breakfast
2     2020-11-11 0 days 10:20:00 0 days 10:45:00  breakfast
3     2020

In [8]:
tmp = pd.to_datetime("2022-02-01")
day = pd.to_timedelta(1, "day")
print(tmp+day)
print(len(df["Date"].unique()))

2022-02-02 00:00:00
536


In [9]:
## Make sure all dates are present
dates = [str(x)[:10] for x in df["Date"].unique()]
print(dates[:10], "\n")
temp = "2020-11-11"
for date in dates[1:]:
    #print(date, temp)
    if(pd.to_datetime(temp)+day != pd.to_datetime(date)):
        print("Missing:", pd.to_datetime(temp)+day, pd.to_datetime(date))
        #break
    temp = date
    #break

['2020-11-11', '2020-11-12', '2020-11-13', '2020-11-14', '2020-11-15', '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19', '2020-11-20'] 



In [10]:
## Now look at Det1, keep "ok" values only 
# df["Det1"].unique()
## Remove words that have no definition
df

,Date,Starttime,Endtime,Activity
0,2020-11-11,0 days 08:30:00,0 days 09:30:00,break
1,2020-11-11,0 days 09:30:00,0 days 10:20:00,breakfast
2,2020-11-11,0 days 10:20:00,0 days 10:45:00,breakfast
3,2020-11-11,0 days 10:45:00,0 days 12:20:00,planning
4,2020-11-11,0 days 12:20:00,0 days 12:30:00,108
...,...,...,...,...
14216,2022-04-30,0 days 20:15:00,0 days 21:50:00,break
14217,2022-04-30,0 days 21:50:00,0 days 22:20:00,break
14218,2022-04-30,0 days 22:20:00,0 days 23:05:00,break
14219,2022-04-30,0 days 23:05:00,0 days 23:15:00,162


In [11]:
## Turn time blocks into multiple 5-minute instants
i = 0
with open('time-log-clean-anon.csv', 'w') as f:
    f.write("Time,Activity\n")
print(df.shape)
for rowtup in tqdm(df.iterrows()):
    if(i%100==0):
        print("Row:",i)
    rowind = rowtup[0]
    data = rowtup[1]
#     print(rowind, "\n", data)
#     print("\nCan add datetime64 + timedelta64:", data[0]+data[1])
#     print("\nCan divide times:", (data[2]-data[1])/5)
#     print("\nCan cast as int:", ((data[2]-data[1])/5/60).seconds)
    num_reps = ((data["Endtime"]-data["Starttime"])/5/60).seconds
    if(data["Endtime"]-data["Starttime"] != abs(data["Endtime"]-data["Starttime"])):
        print("====", rowtup)
    for rep in range(0, num_reps):
        mins = pd.to_timedelta("00:05:00")*rep
        rowdict = {"Time":data[0]+data[1]+mins,    # Date + Start_time + 5_Minute_Increments
                   "Activity":data["Activity"]}
                   #"Detail_1":data["Det1"], 
                   #"Detail_2":data["Det2"], 
                   #"Detail_3":data["Det3"]}
        
        with open('time-log-clean-anon.csv', 'a') as f:
            f.write("{},{}\n".format(rowdict["Time"], rowdict["Activity"]))
    i += 1
    
    #DEBUG: there seem to be no middle-of-night data? 
    # Starts at 2021-04-13
#     if(str(data.Date)[:10]=="2021-04-15" and i>=200):
#         print(data[0]+data[1]+mins, str(data.Date))
#         break
#### Turns out, it took a long time on certain days because an end
#### time was earlier than a start time, so it went all the way around

(14221, 4)


16it [00:00, 54.11it/s]

Row: 0


164it [00:00, 295.45it/s]

Row: 100


...

14036it [00:35, 347.97it/s]

Row: 14000


14164it [00:35, 388.32it/s]

Row: 14100
Row: 14200


14221it [00:35, 395.27it/s]


In [12]:
print(abs(data["Starttime"]-data["Endtime"]))
print(data["Starttime"]-data["Endtime"])
print(abs(data["Endtime"]-data["Starttime"]))
print(data["Endtime"]-data["Starttime"])

0 days 00:40:00
-1 days +23:20:00
0 days 00:40:00
0 days 00:40:00


In [13]:
df2 = pd.read_csv("time-log-clean-anon.csv")
print(df2.shape)

(96304, 2)


In [14]:
df2["Activity"].value_counts()
# (hours of break)/(total hours) for each day
# ((blocks*5/60)/(525? days*24))*24

break        34571
help         10119
career        9185
dinner        4727
lunch         3382
planning      3350
exercise      2669
131b          2390
bike          1977
171           1857
breakfast     1811
prep          1748
137           1603
141a          1408
141b          1322
154a          1302
appt          1256
131a          1247
104           1110
135           1079
167           1068
010            985
106            937
122a           909
141c           871
108            787
162            421
330            397
174            378
302            372
165a           251
160            161
168            153
fys             99
131c            94
CLA             93
walk            87
UDCE            52
130             45
141             20
bus             11
Name: Activity, dtype: int64